In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt


# 定义模型类
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(6035, 50)  # 调整输入大小
        self.fc2 = nn.Linear(50, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 16)
        self.fc5 = nn.Linear(16, 16)
        self.fc6 = nn.Linear(16, 16)
        self.fc7 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.relu(self.fc6(x))
        x = self.sigmoid(self.fc7(x))
        return x


def reset_weights(m):
    """
    重置模型参数
    """
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()


# 导入模型 ./models/model_10_59.pth

model = SimpleNN()
model.load_state_dict(torch.load('./models/model_10_59.pth'))
model.eval()



SimpleNN(
  (fc1): Linear(in_features=6035, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=16, bias=True)
  (fc6): Linear(in_features=16, out_features=16, bias=True)
  (fc7): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [16]:
from androguard.core.apk import APK
import torch
import numpy as np

# 模型输入的参数
train_data = pd.read_csv('./dataset/combined_df.csv')
# 去第一行为params
params = train_data.iloc[:, 1:].to_dict('list')


def predict(path):
    apk = APK(path)
    permissions = apk.get_permissions()
    print(permissions)
    # permissions转换为矩阵，表头为param
    input = []
    for p in params:
        if p in permissions:
            input.append(1)
        else:
            input.append(0)

    # 加载模型

    # 预测结果
    with torch.no_grad():
        x = torch.tensor(np.array(input).astype(np.float32))
        y_pred = model(x)
        y_pred = y_pred.item()
        print("预测结果为：", ["安全", "危险"][int(y_pred > 0.5)])


In [17]:
predict("AZCRcpgMh6wn_HCy2KmJ.apk")

2024-07-16 16:53:34.481 | INFO     | androguard.core.apk:_apk_analysis:358 - Starting analysis on AndroidManifest.xml
2024-07-16 16:53:34.485 | DEBUG    | androguard.core.axml:__init__:966 - AXMLPrinter
2024-07-16 16:53:34.485 | DEBUG    | androguard.core.axml:__init__:396 - AXMLParser
2024-07-16 16:53:34.485 | DEBUG    | androguard.core.axml:__init__:421 - FIRST HEADER <ARSCHeader idx='0x00000000' type='3' header_size='8' size='8912'>
2024-07-16 16:53:34.486 | DEBUG    | androguard.core.axml:__init__:461 - STRING_POOL <ARSCHeader idx='0x00000008' type='1' header_size='28' size='4272'>
2024-07-16 16:53:34.486 | DEBUG    | androguard.core.axml:is_valid:488 - True
2024-07-16 16:53:34.486 | DEBUG    | androguard.core.axml:_do_next:506 - M_EVENT -1
2024-07-16 16:53:34.487 | DEBUG    | androguard.core.axml:_do_next:522 - NEXT HEADER <ARSCHeader idx='0x000010b8' type='384' header_size='8' size='124'>
2024-07-16 16:53:34.487 | DEBUG    | androguard.core.axml:_do_next:531 - AXML contains a RES

['android.permission.READ_MEDIA_IMAGES', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.WAKE_LOCK', 'android.permission.REQUEST_INSTALL_PACKAGES', 'android.permission.ACCESS_WIFI_STATE', 'android.permission.INTERNET', 'android.permission.VIBRATE', 'cn.vpszjw.jd.permission.YM_APP', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.READ_MEDIA_AUDIO', 'android.permission.CAMERA', 'android.permission.READ_MEDIA_VIDEO']
预测结果为： 危险
